# Advanced RAG Techniques with Qdrant

**Author:** Advanced RAG Implementation Guide

---

## 📋 Overview

This notebook provides a **comprehensive reference implementation** of advanced RAG (Retrieval-Augmented Generation) techniques. It demonstrates how to build sophisticated retrieval pipelines that go beyond basic vector similarity search, enabling more accurate and contextually rich AI applications.

### 🎯 Learning Objectives

By the end of this notebook, you will understand:
- How to implement multi-level document chunking strategies
- Techniques for improving retrieval relevance through reranking
- Advanced query processing methods (expansion, fusion, planning)
- Evaluation methodologies for RAG systems

---

## 🔑 Key Features

This notebook covers the following **advanced RAG techniques**:

### 1. **Parent/Child Chunking** via `ParentDocumentRetriever`
   - ✅ Enables retrieving precise passages while maintaining broader context
   - ✅ Improves both retrieval precision and generation quality
   - ✅ Reduces context fragmentation in final answers

### 2. **LLM-based Reranking**
   - ✅ Uses an LLM to score and rerank retrieved documents
   - ✅ Improves relevance of final context passed to the generator
   - ✅ Filters out irrelevant documents before generation

### 3. **RAG-Fusion** (Query Expansion + Reciprocal Rank Fusion)
   - ✅ Generates multiple query reformulations
   - ✅ Merges results using Reciprocal Rank Fusion (RRF) algorithm
   - ✅ Significantly improves recall on heterogeneous corpora
   - ✅ Handles ambiguous or multi-faceted queries effectively

### 4. **Query Planning** (Minimal Plan-and-Execute)
   - ✅ Decomposes complex questions into atomic sub-questions
   - ✅ Executes each step sequentially with appropriate tools
   - ✅ Synthesizes results into a coherent final answer
   - ✅ Enables multi-step reasoning workflows

### 5. **Evaluation Framework** (LLM-as-a-Judge)
   - ✅ Evaluates answer quality using an LLM judge
   - ✅ Measures factual accuracy, completeness, and clarity
   - ✅ Includes comparative analysis between different pipeline configurations
   - ✅ Provides quantitative metrics for pipeline optimization

---

## 📦 Prerequisites

### Required Environment Variables
- `OPENAI_API_KEY` - Your OpenAI API key (must be set)

### Required Python Packages
```bash
pip install langchain langchain-openai langchain-community qdrant-client tiktoken ragas pypdf networkx rank-bm25
```

**Note:** The notebook includes an installation cell, but ensure you have these packages installed before running.


## 📦 Installation

Install the required packages for this notebook. Run this cell if you haven't already installed the dependencies.


In [1]:
# !pip install -qU langchain langchain-openai langchain-community qdrant-client tiktoken ragas pypdf networkx rank-bm25

## 🔧 Environment Setup

Verify that your OpenAI API key is configured. This notebook requires an active `OPENAI_API_KEY` environment variable to function.


In [2]:
import os

# Try to load from .env file if available (optional)
try:
    from dotenv import load_dotenv
    load_dotenv()
except ImportError:
    pass  # dotenv not installed, that's okay

assert os.getenv("OPENAI_API_KEY"), "⚠️ Please set OPENAI_API_KEY in your environment!"
print("✅ OK: OPENAI_API_KEY detected")


✅ OK: OPENAI_API_KEY detected


## 📚 Imports and Dependencies

Import all necessary libraries for building advanced RAG pipelines:
- **LangChain components**: Document handling, text splitters, embeddings, vector stores
- **Qdrant**: Vector database for storing embeddings
- **OpenAI**: Embeddings and LLM models
- **Core utilities**: Type hints, JSON handling, itertools for result fusion


In [3]:
from typing import List, Dict, Any
import json
import itertools
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import Qdrant
from qdrant_client import QdrantClient
from langchain_classic.retrievers.parent_document_retriever import ParentDocumentRetriever
from langchain_classic.storage import InMemoryStore
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate

## 1️⃣ Example Corpus Setup

Create a small demonstration corpus with sample documents about RAG techniques. This corpus will be used throughout the notebook to demonstrate various advanced RAG methods.

**What this section does:**
- Creates sample documents about RAG techniques
- Prepares text splitters for parent and child chunking
- Sets up the document structure for the parent/child retriever


In [4]:
raws = [
    """Mr. Jonas, a fluffy orange cat, decided he was tired of just napping all day. He applied for a promotion at the local office. Unfortunately, during the interview, he fell asleep on the keyboard, accidentally sending a company-wide email that said, 'Meow if you approve!' Surprisingly, the staff loved it, thinking it was a new motivational strategy. Mr. Jonas got the promotion, now supervising the office naps. Soon, he implemented 'cat breaks' every hour, where employees were encouraged to stretch, purr softly, or chase imaginary laser pointers. Productivity oddly skyrocketed, and the office became famous for its feline-inspired efficiency seminars. Mr. Jonas even started a newsletter titled 'The Purr-fect Manager,' sharing tips on balancing work and naps.""",
    """Jenny bought a toaster from a mysterious yard sale, not realizing it was a time machine. Every time she toasted bread, it popped up with tiny messages from the future. One morning, the toast said, 'Beware the dancing squirrels at 3 PM!' She ignored it, and sure enough, a flash mob of squirrels waltzed through her garden, all wearing tiny top hats. Jenny now always eats her breakfast with a helmet. Eventually, she discovered a secret lever inside the toaster that allowed her to send messages back in time. She used it to warn her past self about spilled coffee, missed deadlines, and unexpected squirrel invasions. Breakfast became an adventure, with toast revealing mysteries of the universe and sometimes giving advice on what socks to wear for luck.""",
    """In the small town of Noodleton, spaghetti rained from the sky for no reason at all. Mayor Luigi tried to stop it with umbrellas, but the noodles were relentless. One ambitious dog named Max decided to chase the falling spaghetti like a game. Soon, the whole town joined, slipping, sliding, and twirling in a chaotic noodle dance-off. By evening, everyone agreed: it was the tastiest disaster in history. Over the next few days, chefs experimented with sky-spaghetti recipes, inventing noodle ice cream, spaghetti smoothies, and pasta hats. Scientists tried to figure out the phenomenon but concluded it was 'spaghetti magic.' Max became the town hero, leading annual 'noodle festivals,' and Noodleton transformed into the world's first pasta-themed tourist destination."""
]

docs = [Document(page_content=t, metadata={"source": "demo", "idx": i}) for i, t in enumerate(raws)]

parent_splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=80)
child_splitter = RecursiveCharacterTextSplitter(chunk_size=250, chunk_overlap=40)
parents = parent_splitter.split_documents(docs)
children = []  # Children will be generated by the retriever; we still prepare the structure.

## 2️⃣ Parent/Child Chunking Implementation

The **ParentDocumentRetriever** implements a sophisticated two-level chunking strategy that addresses the classic trade-off between retrieval precision and context completeness.

#### How It Works

- **Child chunks** (smaller, ~250 tokens)
  - Used for initial retrieval via vector similarity search
  - Provides fine-grained matching for precise passage retrieval
  - Better semantic alignment with specific query aspects
  - Enables finding needle-in-haystack information
  
- **Parent chunks** (larger, ~800 tokens)
  - Retrieved automatically when a child chunk matches
  - Provides broader context around the matched passage
  - Reduces context fragmentation and improves answer coherence
  - Maintains narrative flow and related information

#### When to Use

✅ **Ideal for:**
- Documents with dense information where context matters
- Long-form content (articles, reports, documentation)
- Scenarios requiring both precision and completeness
- Applications where answer quality depends on surrounding context

❌ **Less suitable for:**
- Very short documents or FAQ-style content
- Real-time systems with strict latency requirements
- Simple Q&A where exact matches are sufficient


**What this section does:**
- Initializes OpenAI embeddings (`text-embedding-3-small`)
- Creates a Qdrant vector store (in-memory for this demo)
- Sets up the `ParentDocumentRetriever` with:
  - **Child splitter**: Small chunks (~250 tokens) for precise retrieval
  - **Parent splitter**: Larger chunks (~800 tokens) for context
- Indexes the documents using the parent/child strategy

**Key Concept:** The retriever stores child chunks in the vector store for retrieval, but returns parent chunks to provide broader context.


In [5]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# Create an empty Qdrant vectorstore for ParentDocumentRetriever
# We'll use in-memory storage with a unique collection name
# ParentDocumentRetriever will handle adding documents to the vectorstore
client = QdrantClient(location=":memory:")
collection_name = "parent_child_docs"

# Create the vectorstore - we need to create it with from_texts to initialize the collection
# Using an empty list to create the collection structure
from qdrant_client.models import Distance, VectorParams
try:
    # Try to get the collection, if it doesn't exist, create it
    client.get_collection(collection_name)
except:
    # Collection doesn't exist, create it
    client.create_collection(
        collection_name=collection_name,
        vectors_config=VectorParams(
            size=1536,  # text-embedding-3-small dimension
            distance=Distance.COSINE
        )
    )

# Now create the vectorstore
vectorstore = Qdrant(
    client=client,
    collection_name=collection_name,
    embeddings=embeddings
)

store = InMemoryStore()
retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
)
retriever.add_documents(docs)

/var/folders/05/3bcz94ds2xlfpypg943txflc0000gn/T/ipykernel_32652/2257423038.py:26: LangChainDeprecationWarning: The class `Qdrant` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the `langchain-qdrant package and should be used instead. To use it run `pip install -U `langchain-qdrant` and import as `from `langchain_qdrant import Qdrant``.
  vectorstore = Qdrant(


## 3️⃣ LLM-based Reranking

Implement reranking using an LLM to score document relevance. This improves retrieval quality by filtering out irrelevant documents before generation.

**What this section does:**
- Defines a scoring prompt that asks the LLM to rate document relevance (0-100)
- Implements `rerank_llm()` function that:
  1. Scores each document for relevance to the query
  2. Sorts documents by score
  3. Returns the top-k most relevant documents

**Benefits:**
- Better document selection than pure vector similarity
- Reduces noise in the context passed to the generator
- Improves answer quality by focusing on truly relevant passages

**Note:** This is computationally expensive as it requires one LLM call per document. Consider batching or using dedicated reranking models for production.


In [6]:
from operator import itemgetter

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

def score_prompt():
    return ChatPromptTemplate.from_messages([
        ("system", "You are an evaluator. Rate from 0 to 100 the relevance of the PASSAGE for answering the QUESTION. "
                   "Respond only with an integer."),
        ("human", "QUESTION:\n{question}\n\nPASSAGE:\n{passage}\n\nScore (0-100):")
    ])

async def score_passage_async(passage: str, question: str):
    # Use LCEL pattern for LangChain 1.0 compatibility
    prompt_template = score_prompt()
    chain = prompt_template | llm
    resp = await chain.ainvoke({"question": question, "passage": passage})
    try:
        return int(''.join(ch for ch in resp.content if ch.isdigit())[:3] or 0)
    except Exception:
        return 0

def rerank_llm(passages: List[Document], question: str, k: int = 3) -> List[Document]:
    # Synchronous call for simplicity (can be async with asyncio.gather)
    scored = []
    prompt_template = score_prompt()
    # Use LCEL pattern for LangChain 1.0 compatibility
    chain = prompt_template | llm
    for d in passages:
        resp = chain.invoke({"question": question, "passage": d.page_content[:1500]})
        try:
            s = int(''.join(ch for ch in resp.content if ch.isdigit())[:3] or 0)
        except Exception:
            s = 0
        scored.append((s, d))
    scored.sort(key=lambda x: x[0], reverse=True)
    return [d for _, d in scored[:k]]

## 🔄 RAG Pipeline Funnel: From BM25 to Final Reranking

This section demonstrates the **funnel operation** of a multi-stage RAG pipeline, showing how documents are progressively filtered and refined from initial frequency-based retrieval to final reranking.

#### The Funnel Concept

A RAG funnel progressively narrows down the document set through multiple retrieval stages:

```
┌─────────────────────────────────────────┐
│  Stage 1: BM25 (Frequency-based)        │
│  → Broad retrieval using term frequency │
│  → Returns: ~20-30 documents            │
└─────────────────────────────────────────┘
              ↓
┌─────────────────────────────────────────┐
│  Stage 2: Vector Similarity (Semantic)   │
│  → Semantic matching using embeddings   │
│  → Returns: ~8-10 documents             │
└─────────────────────────────────────────┘
              ↓
┌─────────────────────────────────────────┐
│  Stage 3: LLM Reranking                  │
│  → Contextual relevance scoring          │
│  → Returns: ~3-5 final documents         │
└─────────────────────────────────────────┘
```

#### Why Use a Funnel?

**Advantages:**
- ✅ **Efficiency**: BM25 is fast and catches keyword matches
- ✅ **Recall**: Initial broad retrieval ensures nothing is missed
- ✅ **Precision**: Each stage refines results for better relevance
- ✅ **Cost Optimization**: Expensive LLM reranking only on top candidates
- ✅ **Hybrid Approach**: Combines lexical (BM25) and semantic (vector) signals

**Trade-offs:**
- ⚠️ More complex pipeline with multiple stages
- ⚠️ Requires tuning of k values at each stage
- ⚠️ May filter out relevant documents if stages are too aggressive

**What this section does:**
- Implements BM25 retrieval using frequency-based term matching
- Demonstrates the funnel operation with visual output
- Shows document counts and snippets at each stage
- Compares results across different retrieval methods


In [7]:
from rank_bm25 import BM25Okapi
import re
from collections import Counter

# Prepare documents for BM25 (need tokenized text)
def tokenize(text: str) -> List[str]:
    """Simple tokenization for BM25"""
    # Convert to lowercase and split on whitespace/punctuation
    tokens = re.findall(r'\b\w+\b', text.lower())
    return tokens

# Get all child chunks from the retriever for BM25 indexing
# We'll use the parent documents and split them for BM25
all_docs_for_bm25 = []
for doc in docs:
    # Split each document into smaller chunks for BM25
    chunks = child_splitter.split_documents([doc])
    all_docs_for_bm25.extend(chunks)

# Tokenize all documents for BM25
tokenized_docs = [tokenize(doc.page_content) for doc in all_docs_for_bm25]

# Create BM25 index
bm25 = BM25Okapi(tokenized_docs)

def retrieve_bm25(query: str, k: int = 20) -> List[Document]:
    """
    Retrieve documents using BM25 (frequency-based approach).
    
    BM25 (Best Matching 25) is a ranking function that scores documents
    based on term frequency (TF) and inverse document frequency (IDF).
    It's particularly good at matching exact keywords and phrases.
    """
    query_tokens = tokenize(query)
    scores = bm25.get_scores(query_tokens)
    
    # Get top k documents
    top_indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:k]
    
    results = []
    for idx in top_indices:
        if scores[idx] > 0:  # Only include documents with non-zero scores
            results.append(all_docs_for_bm25[idx])
    
    return results

def retrieve_vector(query: str, k: int = 10) -> List[Document]:
    """
    Retrieve documents using vector similarity (semantic search).
    
    Uses the parent/child retriever which performs semantic matching
    using embeddings. This captures meaning and context beyond keywords.
    """
    return retriever.invoke(query)[:k]

def demonstrate_rag_funnel(question: str, bm25_k: int = 20, vector_k: int = 8, rerank_k: int = 3):
    """
    Demonstrate the RAG funnel operation showing progressive filtering.
    
    Pipeline:
    1. BM25 retrieval (frequency-based) → broad set
    2. Vector similarity (semantic) → refined set  
    3. LLM reranking → final top documents
    """
    print("\n" + "="*70)
    print("🔄 RAG PIPELINE FUNNEL DEMONSTRATION")
    print("="*70)
    print(f"\n📌 Query: {question}\n")
    
    # Stage 1: BM25 (Frequency-based retrieval)
    print("─"*70)
    print("📊 STAGE 1: BM25 (Frequency-based Retrieval)")
    print("─"*70)
    print(f"Method: Term frequency + Inverse document frequency (TF-IDF variant)")
    print(f"Approach: Lexical matching - finds documents with query keywords")
    print(f"Retrieval size: Top {bm25_k} documents\n")
    
    bm25_results = retrieve_bm25(question, k=bm25_k)
    print(f"✅ Retrieved {len(bm25_results)} documents via BM25")
    print(f"\nTop 5 BM25 results (showing snippets):")
    for i, doc in enumerate(bm25_results[:5], 1):
        snippet = doc.page_content[:120].replace("\n", " ") + "..."
        print(f"  {i}. {snippet}")
    
    # Stage 2: Vector Similarity (Semantic retrieval)
    print("\n" + "─"*70)
    print("🔍 STAGE 2: Vector Similarity (Semantic Retrieval)")
    print("─"*70)
    print(f"Method: Embedding-based cosine similarity")
    print(f"Approach: Semantic matching - understands meaning and context")
    print(f"Retrieval size: Top {vector_k} documents\n")
    
    vector_results = retrieve_vector(question, k=vector_k)
    print(f"✅ Retrieved {len(vector_results)} documents via vector similarity")
    print(f"\nTop {min(5, len(vector_results))} vector results (showing snippets):")
    for i, doc in enumerate(vector_results[:5], 1):
        snippet = doc.page_content[:120].replace("\n", " ") + "..."
        print(f"  {i}. {snippet}")
    
    # Stage 3: Reranking
    print("\n" + "─"*70)
    print("🏆 STAGE 3: LLM-based Reranking")
    print("─"*70)
    print(f"Method: LLM contextual relevance scoring")
    print(f"Approach: Deep understanding of query-document relationship")
    print(f"Input: Top {len(vector_results)} from vector search")
    print(f"Output: Top {rerank_k} most relevant documents\n")
    
    # Rerank the vector results
    reranked_results = rerank_llm(vector_results, question, k=rerank_k)
    print(f"✅ Reranked to {len(reranked_results)} final documents")
    print(f"\nFinal top {len(reranked_results)} reranked results:")
    for i, doc in enumerate(reranked_results, 1):
        snippet = doc.page_content[:150].replace("\n", " ") + "..."
        print(f"  {i}. {snippet}")
    
    # Summary
    print("\n" + "="*70)
    print("📈 FUNNEL SUMMARY")
    print("="*70)
    print(f"Stage 1 (BM25):        {len(bm25_results):3d} documents → Frequency-based keyword matching")
    print(f"Stage 2 (Vector):      {len(vector_results):3d} documents → Semantic similarity filtering")
    print(f"Stage 3 (Reranking):   {len(reranked_results):3d} documents → Contextual relevance scoring")
    print(f"\nReduction: {len(bm25_results)} → {len(vector_results)} → {len(reranked_results)}")
    print(f"Final precision: {len(reranked_results)}/{len(bm25_results)} = {len(reranked_results)/len(bm25_results)*100:.1f}% of initial set")
    print("="*70 + "\n")
    
    return {
        "bm25": bm25_results,
        "vector": vector_results,
        "reranked": reranked_results
    }

# Example demonstration
results = demonstrate_rag_funnel("What happened to Mr. Jonas at the office?")



🔄 RAG PIPELINE FUNNEL DEMONSTRATION

📌 Query: What happened to Mr. Jonas at the office?

──────────────────────────────────────────────────────────────────────
📊 STAGE 1: BM25 (Frequency-based Retrieval)
──────────────────────────────────────────────────────────────────────
Method: Term frequency + Inverse document frequency (TF-IDF variant)
Approach: Lexical matching - finds documents with query keywords
Retrieval size: Top 20 documents

✅ Retrieved 12 documents via BM25

Top 5 BM25 results (showing snippets):
  1. became an adventure, with toast revealing mysteries of the universe and sometimes giving advice on what socks to wear fo...
  2. Mr. Jonas, a fluffy orange cat, decided he was tired of just napping all day. He applied for a promotion at the local of...
  3. he implemented 'cat breaks' every hour, where employees were encouraged to stretch, purr softly, or chase imaginary lase...
  4. sending a company-wide email that said, 'Meow if you approve!' Surprisingly, the staff lov

### 🔀 Hybrid Retrieval: Combining BM25 and Vector Search

In practice, many production RAG systems combine BM25 and vector search results for optimal performance. This hybrid approach leverages both lexical (keyword) and semantic (meaning) signals.

**Benefits of Hybrid Approach:**
- ✅ **Better Recall**: Catches both exact keyword matches (BM25) and semantic matches (vector)
- ✅ **Robustness**: Handles queries that benefit from either approach
- ✅ **Complementary Signals**: BM25 excels at specific terms, vectors excel at concepts

**Common Fusion Strategies:**
1. **Reciprocal Rank Fusion (RRF)**: Merge rankings from both methods
2. **Weighted Combination**: Weighted average of BM25 and vector scores
3. **Union then Rerank**: Take union of results, then rerank

Let's demonstrate a hybrid approach using RRF to combine BM25 and vector results:


In [8]:
def hybrid_retrieval_bm25_vector(query: str, bm25_k: int = 15, vector_k: int = 10, final_k: int = 8):
    """
    Hybrid retrieval combining BM25 and vector search using Reciprocal Rank Fusion.
    
    This demonstrates how to combine frequency-based and semantic retrieval
    for better overall performance.
    """
    print("\n" + "="*70)
    print("🔀 HYBRID RETRIEVAL: BM25 + Vector Search")
    print("="*70)
    print(f"\n📌 Query: {query}\n")
    
    # Get results from both methods
    bm25_results = retrieve_bm25(query, k=bm25_k)
    vector_results = retrieve_vector(query, k=vector_k)
    
    print(f"📊 BM25 Results:    {len(bm25_results)} documents")
    print(f"🔍 Vector Results:  {len(vector_results)} documents")
    
    # Combine using RRF
    # Create a simple RRF implementation for two result sets
    def simple_rrf(list1: List[Document], list2: List[Document], k: int = 60) -> List[Document]:
        scores = {}
        seen = set()
        
        # Score documents from first list (BM25)
        for rank, doc in enumerate(list1, start=1):
            key = (doc.metadata.get("source"), doc.metadata.get("idx"), doc.page_content[:50])
            if key not in seen:
                scores[key] = scores.get(key, 0) + 1.0 / (k + rank)
                seen.add(key)
        
        # Score documents from second list (Vector)
        for rank, doc in enumerate(list2, start=1):
            key = (doc.metadata.get("source"), doc.metadata.get("idx"), doc.page_content[:50])
            if key not in seen:
                scores[key] = scores.get(key, 0) + 1.0 / (k + rank)
                seen.add(key)
            else:
                # Document appears in both lists - boost its score
                scores[key] += 1.0 / (k + rank)
        
        # Sort by score
        ranked = sorted(scores.items(), key=lambda kv: kv[1], reverse=True)
        
        # Reconstruct documents
        all_docs = list1 + list2
        result = []
        seen_keys = set()
        for key, score in ranked[:final_k]:
            for doc in all_docs:
                doc_key = (doc.metadata.get("source"), doc.metadata.get("idx"), doc.page_content[:50])
                if doc_key == key and doc_key not in seen_keys:
                    result.append(doc)
                    seen_keys.add(doc_key)
                    break
        
        return result
    
    hybrid_results = simple_rrf(bm25_results, vector_results, k=60)
    
    print(f"🔗 Hybrid Results:  {len(hybrid_results)} documents (after RRF fusion)\n")
    
    print("Top hybrid results (BM25 + Vector fusion):")
    for i, doc in enumerate(hybrid_results[:5], 1):
        snippet = doc.page_content[:120].replace("\n", " ") + "..."
        print(f"  {i}. {snippet}")
    
    print("\n" + "─"*70)
    print("📈 Comparison Summary")
    print("─"*70)
    print(f"BM25 only:        {len(bm25_results)} documents")
    print(f"Vector only:      {len(vector_results)} documents")
    print(f"Hybrid (RRF):     {len(hybrid_results)} documents")
    print("="*70 + "\n")
    
    return hybrid_results

# Demonstrate hybrid retrieval
hybrid_docs = hybrid_retrieval_bm25_vector("What unusual events happened in these stories?")



🔀 HYBRID RETRIEVAL: BM25 + Vector Search

📌 Query: What unusual events happened in these stories?

📊 BM25 Results:    4 documents
🔍 Vector Results:  2 documents
🔗 Hybrid Results:  5 documents (after RRF fusion)

Top hybrid results (BM25 + Vector fusion):
  1. In the small town of Noodleton, spaghetti rained from the sky for no reason at all. Mayor Luigi tried to stop it with um...
  2. became an adventure, with toast revealing mysteries of the universe and sometimes giving advice on what socks to wear fo...
  3. Jenny bought a toaster from a mysterious yard sale, not realizing it was a time machine. Every time she toasted bread, i...
  4. falling spaghetti like a game. Soon, the whole town joined, slipping, sliding, and twirling in a chaotic noodle dance-of...
  5. discovered a secret lever inside the toaster that allowed her to send messages back in time. She used it to warn her pas...

──────────────────────────────────────────────────────────────────────
📈 Comparison Summary
──────

## 4️⃣ RAG-Fusion Implementation

Implement **RAG-Fusion**, a technique that combines query expansion with Reciprocal Rank Fusion (RRF) to significantly improve recall. **RAG-Fusion** is a powerful technique that combines query expansion with result fusion to significantly improve recall, especially on heterogeneous or complex corpora.

#### Core Components

**1. Query Expansion**
- Generates diverse reformulations of the original question
- Captures different phrasings, perspectives, and semantic variations
- Helps retrieve documents that might be missed by the original query
- Example: "What is RAG?" → ["Explain RAG", "How does retrieval-augmented generation work?", "Describe RAG systems"]

**2. Reciprocal Rank Fusion (RRF)**
- Merges results from multiple queries using a mathematical formula
- **Formula**: `score = Σ(1 / (k + rank))` across all query results
- Documents appearing in multiple result sets receive higher scores
- Naturally promotes consensus documents (documents relevant to multiple query variations)
- The constant `k` (typically 60) prevents division by zero and smooths the ranking

#### When to Use

✅ **Particularly effective for:**
- Heterogeneous corpora with varied document types
- Ambiguous queries that could be interpreted multiple ways
- Multi-domain knowledge bases
- Scenarios where recall is more important than precision
- Complex questions requiring information from multiple perspectives

❌ **May be overkill for:**
- Simple, well-defined queries
- Homogeneous document collections
- Systems with strict latency constraints
- Very specific, unambiguous questions


**What this section does:**

### Query Expansion
- `query_expansions()`: Generates multiple reformulations of the original query
- Uses an LLM to create diverse phrasings that capture different aspects of the question
- Helps retrieve documents that might be missed by the original query

### Reciprocal Rank Fusion (RRF)
- `reciprocal_rank_fusion()`: Merges results from multiple queries
- **Formula**: `score = Σ(1 / (k + rank))` across all query result sets
- Documents appearing in multiple result sets get higher scores
- Naturally promotes consensus documents (documents relevant to multiple query variations)

### Search Function
- `search_once()`: Performs a single retrieval using the parent/child retriever

**Benefits:**
- Significantly improves recall on heterogeneous corpora
- Handles ambiguous queries effectively
- Captures information from multiple perspectives


In [9]:
def query_expansions(question: str, n: int = 4) -> List[str]:
    tmpl = ChatPromptTemplate.from_messages([
        ("system", "You are an assistant who proposes short and diverse reformulations of a question."),
        ("human", "Question: {q}\nGive {n} different reformulations, one per line, without numbers.")
    ])
    # Use LCEL pattern for LangChain 1.0 compatibility
    chain = tmpl | llm
    resp = chain.invoke({"q": question, "n": n})
    lines = [ln.strip("- •* ").strip() for ln in resp.content.splitlines() if ln.strip()]
    return lines[:n] if lines else [question]

def search_once(q: str, k: int = 6) -> List[Document]:
    # uses the parent/child retriever (LangChain 1.0 uses invoke() instead of get_relevant_documents())
    return retriever.invoke(q)[:k]

def reciprocal_rank_fusion(list_of_results: List[List[Document]], k: int = 60) -> List[Document]:
    # RRF: score = sum(1/(k + rank(d)))
    scores = {}
    for results in list_of_results:
        seen = {}
        for rank, d in enumerate(results, start=1):
            key = (d.metadata.get("source"), d.metadata.get("idx"), d.page_content[:50])
            if key in seen: 
                continue
            seen[key] = True
            scores[key] = scores.get(key, 0) + 1.0 / (k + rank)
    ranked = sorted(scores.items(), key=lambda kv: kv[1], reverse=True)
    # reconstruct Documents from keys; here we return the first existing results
    flat = list(itertools.chain.from_iterable(list_of_results))
    out = []
    for key, _ in ranked:
        for d in flat:
            if (d.metadata.get("source"), d.metadata.get("idx"), d.page_content[:50]) == key:
                out.append(d)
                break
    return out

## 5️⃣ Advanced RAG Pipeline

Combine all advanced techniques into a single, powerful RAG pipeline that demonstrates the full potential of advanced retrieval methods.

**What this section does:**
- `rag_advanced_pipeline()` integrates:
  1. **Query Expansion**: Generates 4 reformulations of the query
  2. **Multi-query Retrieval**: Searches with original + expanded queries
  3. **RAG-Fusion**: Merges results using Reciprocal Rank Fusion
  4. **Reranking**: LLM-based reranking of top candidates
  5. **Generation**: Final answer generation with reranked context

**Pipeline Flow:**
```
Query → Query Expansion → Multi-query Retrieval → RRF Fusion → LLM Reranking → Generation
```

This demonstrates a production-ready advanced RAG system that combines multiple techniques for optimal performance.


In [10]:
def rag_advanced_pipeline(question: str) -> str:
    print("\n" + "="*60)
    print(f"📌 Initial Question:\n{question}")
    print("="*60 + "\n")

    # Expansions
    exps = query_expansions(question, n=4)
    print(f"💡 Query Expansions ({len(exps)}):")
    for i, exp in enumerate(exps, 1):
        print(f"  {i}. {exp}")
    print("-"*60 + "\n")

    # Retrieval
    all_results = [search_once(q, k=6) for q in [question] + exps]
    print(f"🔍 Single Retrieval Results ({len(all_results)} queries):")
    for i, results in enumerate(all_results, 1):
        print(f"  Query {i}: {len(results)} results")
    print("-"*60 + "\n")

    # Fusion
    fused = reciprocal_rank_fusion(all_results, k=60)
    print(f"🔗 Fused Results: {len(fused)} passages (showing top 3):")
    for d in fused[:3]:
        snippet = d.page_content[:150].replace("\n", " ") + ("..." if len(d.page_content) > 150 else "")
        print(f"  - {snippet}")
    print("-"*60 + "\n")

    # Reranking
    reranked = rerank_llm(fused[:8], question, k=3)
    print(f"🏆 Reranked Top {len(reranked)} Results:")
    for i, d in enumerate(reranked, 1):
        snippet = d.page_content[:150].replace("\n", " ") + ("..." if len(d.page_content) > 150 else "")
        print(f"  {i}. {snippet}")
    print("="*60 + "\n")

    # Final generation
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are an assistant who responds concisely by explicitly citing the key ideas from the provided PASSAGES."),
        ("human", "QUESTION:\n{q}\n\nPASSAGES:\n{ctx}\n\nAnswer:")
    ])
    ctx = "\n---\n".join(d.page_content for d in reranked)
    chain = prompt | llm
    resp = chain.invoke({"q": question, "ctx": ctx[:4000]})

    print("✅ Final Answer:")
    print(resp.content)
    return resp.content


# Example call
rag_advanced_pipeline("Explain who are each character of these stories.")


📌 Initial Question:
Explain who are each character of these stories.

💡 Query Expansions (4):
  1. Can you describe the characters in these stories?
  2. Who are the characters in each of these narratives?
  3. Please provide an overview of the characters from these tales.
  4. What can you tell me about the characters in these stories?
------------------------------------------------------------

🔍 Single Retrieval Results (5 queries):
  Query 1: 2 results
  Query 2: 2 results
  Query 3: 2 results
  Query 4: 1 results
  Query 5: 2 results
------------------------------------------------------------

🔗 Fused Results: 2 passages (showing top 3):
  - Jenny bought a toaster from a mysterious yard sale, not realizing it was a time machine. Every time she toasted bread, it popped up with tiny messages...
  - In the small town of Noodleton, spaghetti rained from the sky for no reason at all. Mayor Luigi tried to stop it with umbrellas, but the noodles were ...
------------------------------

"**Characters:**\n\n1. **Mayor Luigi** - The leader of Noodleton who attempts to manage the chaos caused by the spaghetti rain but ultimately embraces the fun of the situation.\n\n2. **Max** - An ambitious dog who turns the falling spaghetti into a game, becoming a town hero and leading the community in celebrating the phenomenon with annual noodle festivals.\n\n3. **Jenny** - A curious individual who unknowingly buys a time-traveling toaster, which provides her with future messages and allows her to send warnings back in time.\n\n4. **Dancing Squirrels** - A whimsical group that disrupts Jenny's life with their unexpected flash mob, adding humor and chaos to her daily routine."

## 6️⃣ Query Planning (Plan-and-Execute)

Implement a **query planning** system that decomposes complex questions into atomic sub-questions and executes them sequentially.
**Query Planning** (plan-and-execute pattern) enables sophisticated multi-step reasoning by breaking down complex questions into manageable atomic sub-tasks.

#### Architecture

**Planning Phase**
- LLM decomposes the question into 2-5 atomic steps
- Each step includes:
  - A description of what needs to be done
  - A specific query to execute
  - An assigned tool (retrieval, calculation, none, etc.)
- Enables multi-step reasoning and tool orchestration

**Execution Phase**
- Each step is executed independently
- Results are collected as "notes" or intermediate findings
- Tools can include:
  - **Retrieval**: Search the knowledge base
  - **Calculation**: Perform computations (use with caution in production!)
  - **None**: Direct reasoning without tools

**Synthesis Phase**
- Final LLM call combines all notes into a coherent answer
- Maintains logical flow and structure
- Integrates information from multiple sources

#### When to Use

✅ **Best for complex questions requiring:**
- Multiple retrieval queries (e.g., "Compare X and Y")
- Sequential reasoning (e.g., "First find A, then use A to find B")
- Integration of different information sources
- Tool usage beyond simple retrieval
- Multi-part analytical questions

❌ **Not necessary for:**
- Simple, single-answer questions
- Direct fact retrieval
- Questions answerable with one retrieval call


**What this section does:**

### Planning Phase
- `simple_planner()`: Uses an LLM to break down complex questions into 2-5 atomic steps
- Each step includes:
  - **step**: Description of what needs to be done
  - **query**: Specific query to execute
  - **tool**: Tool to use (retrieval, calculation, or none)

### Execution Phase
- `execute_plan()`: Executes each step in the plan:
  - **Retrieval tool**: Searches the knowledge base and reranks results
  - **Calculation tool**: Performs computations (⚠️ uses `eval()` - not for production!)
  - **None tool**: Direct reasoning without tools
- Collects results as "notes" from each step

### Synthesis Phase
- Final LLM call synthesizes all notes into a coherent, structured answer
- Maintains logical flow and integrates information from multiple sources

**Use Case:** Best for complex, multi-part questions that require sequential reasoning or multiple information sources.


In [11]:
# 6) Query Planning (minimal plan-and-execute)
plan_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a planner. Decompose the QUESTION into 2–5 atomic steps in JSON format: "
               "[{{\"step\": \"...\", \"query\": \"...\", \"tool\": \"retrieval|calc|none\"}}]"),
    ("human", "QUESTION: {q}")
])

def simple_planner(question: str) -> List[Dict[str, Any]]:
    # Use LCEL pattern for LangChain 1.0 compatibility
    chain = plan_prompt | llm
    resp = chain.invoke({"q": question})
    try:
        plan = json.loads(resp.content)
        if isinstance(plan, list):
            return plan[:5]
    except Exception:
        pass
    return [{"step": "General search", "query": question, "tool": "retrieval"}]

def execute_plan(plan: List[Dict[str, Any]]) -> str:
    notes = []
    for p in plan:
        tool = p.get("tool", "retrieval")
        q = p.get("query", "")
        if tool == "retrieval":
            docs = retriever.invoke(q)[:6]  # LangChain 1.0 uses invoke() instead of get_relevant_documents()
            docs = rerank_llm(docs, q, k=2)
            notes.append("\n".join(d.page_content for d in docs))
        elif tool == "calc":
            # placeholder for external calculation (e.g., Python)
            try:
                res = eval(q)  # ⚠️ DO NOT do this in production (security), controlled demo here
            except Exception:
                res = f"calc_failed({q})"
            notes.append(str(res))
        else:
            notes.append(f"(no tool) {q}")
    # final synthesis
    prompt = ChatPromptTemplate.from_messages([
        ("system", "Synthesize the NOTES below into a clear and structured answer."),
        ("human", "QUESTION: {q}\nNOTES:\n{notes}\nAnswer:")
    ])
    # Use LCEL pattern for LangChain 1.0 compatibility
    chain = prompt | llm
    resp = chain.invoke({
        "q": "; ".join([p.get("query","") for p in plan]), 
        "notes": "\n---\n".join(notes)[:5000]
    })
    return resp.content

question = "Compare main caracter's personality"
plan = simple_planner(question)
print("PLAN:", plan)
print("\nANSWER:\n", execute_plan(plan))

PLAN: [{'step': 'Identify the main character', 'query': 'Who is the main character in the story?', 'tool': 'retrieval'}, {'step': 'List personality traits', 'query': 'What are the personality traits of the main character?', 'tool': 'retrieval'}, {'step': 'Identify another character for comparison', 'query': 'Who is the other character to compare with?', 'tool': 'retrieval'}, {'step': 'List personality traits of the other character', 'query': 'What are the personality traits of the other character?', 'tool': 'retrieval'}, {'step': 'Compare the personality traits', 'query': 'How do the personality traits of the two characters compare?', 'tool': 'none'}]

ANSWER:
 **Main Character: Jenny**

- **Personality Traits:**
  - Adventurous: Jenny embraces the unexpected, as seen when she buys a mysterious toaster.
  - Curious: She explores the capabilities of the toaster, discovering it can send messages from the future.
  - Playful: Jenny turns her breakfast routine into an adventure, enjoying t

## 7️⃣ Evaluation with LLM-as-a-Judge

Implement evaluation using an **LLM-as-a-Judge** approach to assess answer quality.

**✅ Advantages:**
- Flexible evaluation criteria (can assess clarity, completeness, nuance)
- No need for labeled datasets
- Can evaluate open-ended questions
- Understands context and semantic similarity

**❌ Limitations:**
- Non-deterministic (same answer may get different scores)
- Potential bias based on judge model
- Computationally expensive
- May not align with human judgment perfectly


**What this section does:**
- `judge_answer()`: Evaluates answer quality using an LLM judge
- **Criteria evaluated:**
  - ✅ **Factual Accuracy**: Alignment with retrieved passages
  - ✅ **Completeness**: Coverage of important aspects
  - ✅ **Clarity**: How well the answer is structured and explained
- Returns a score (0-100) and detailed feedback

**Example Usage:**
- Generates an answer using the advanced pipeline
- Retrieves relevant context documents
- Judges the answer quality against the context

**Note:** This is a demonstration of LLM-based evaluation. For production, consider using RAGAS framework or human evaluation for more reliable metrics.


In [12]:
judge = ChatOpenAI(model="gpt-4o-mini", temperature=0)
judge_prompt = ChatPromptTemplate.from_messages([
    ("system", "You evaluate the answer. Criteria: factual accuracy vs PASSAGES, completeness, clarity. Give a score 0-100 and 2 remarks."),
    ("human", "QUESTION: {q}\nANSWER: {ans}\nPASSAGES: {ctx}\nScore and feedback:")
])

def judge_answer(question: str, answer: str, ctx_docs: List[Document]) -> str:
    ctx = "\n---\n".join(d.page_content for d in ctx_docs[:5])
    # Use LCEL pattern for LangChain 1.0 compatibility
    chain = judge_prompt | judge
    resp = chain.invoke({"q": question, "ans": answer, "ctx": ctx[:4000]})
    return resp.content

q = "Is Jonas a black cat?"
ans = rag_advanced_pipeline(q)
ctx_docs = retriever.invoke(q)[:5]  # LangChain 1.0 uses invoke() instead of get_relevant_documents()
print(judge_answer(q, ans, ctx_docs))



📌 Initial Question:
Is Jonas a black cat?

💡 Query Expansions (4):
  1. Is Jonas a cat of the black variety?
  2. Is Jonas a feline that is black in color?
  3. Is Jonas classified as a black cat?
  4. Does Jonas belong to the black cat category?
------------------------------------------------------------

🔍 Single Retrieval Results (5 queries):
  Query 1: 1 results
  Query 2: 1 results
  Query 3: 1 results
  Query 4: 1 results
  Query 5: 1 results
------------------------------------------------------------

🔗 Fused Results: 1 passages (showing top 3):
  - Mr. Jonas, a fluffy orange cat, decided he was tired of just napping all day. He applied for a promotion at the local office. Unfortunately, during th...
------------------------------------------------------------

🏆 Reranked Top 1 Results:
  1. Mr. Jonas, a fluffy orange cat, decided he was tired of just napping all day. He applied for a promotion at the local office. Unfortunately, during th...

✅ Final Answer:
No, Jonas is not